In [1]:
model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

In [2]:
from datasets import load_dataset, load_metric

In [29]:
from corus import load_rudrec

json_path = 'data/rudrec_annotated.json'

drugs = list(load_rudrec(json_path))
print(len(drugs))

4809


In [4]:
drugs[0]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

In [5]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [6]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

In [7]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [8]:
print(extract_labels(drugs[0]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [9]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.1, random_state=1)

In [10]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
796,"[И, сейчас, нам, уже, 7, месяцев, ,, аллергии, ни, на, что, нет, .]","[O, O, O, O, O, O, O, B-Finding, O, O, O, O, O]"
2339,"[«, На, дурака, не, нужен, нож, ,, ему, с, три, короба, наврешь, …, », Помните, песню, ?]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1672,"[Взяла, ребенку, для, укрепления, иммунитета, ..., т, ., к, нам, его, педиатр, посоветовала, ,, даже, на, вкус, не, попробовала, .]","[O, O, O, B-DI, I-DI, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [11]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [12]:
from datasets import Dataset, DatasetDict

In [13]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

C:\Python310\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 4328
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 481
    })
})

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [22]:
# show ids
sentence = "Hello, this is one sentence!"
print(tokenizer(sentence), '\n')

# show list of words
tokens1 = tokenizer.tokenize(sentence)
print(tokens1, '\n')

# show ids of the list of words
tokens2 = ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."]
tokenizer(tokens2, is_split_into_words=True)

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]} 

['Hello', ',', 'this', 'is', 'one', 'sentence', '!'] 



{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 7440, 996, 6301, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
example = ner_train[5]
print(example["tokens"])

['Этим', 'средством', 'является', '"', 'Амизон', '"', ',', 'которое', 'хорошо', 'прорекламировано', ',', 'однако', 'такого', 'эффекта', 'как', 'говорят', 'нет', '.']


In [34]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Эти', '##м', 'средство', '##м', 'является', '"', 'А', '##ми', '##зон', '"', ',', 'которое', 'хорошо', 'про', '##рек', '##лами', '##ровано', ',', 'однако', 'такого', 'э', '##ффект', '##а', 'как', 'говорят', 'нет', '.', '[SEP]']


In [35]:
len(example["tags"]), len(tokenized_input["input_ids"])

(18, 29)

In [37]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 9, 9, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17, None]


In [38]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

29 29


In [39]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [40]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 3130, 3374, 23324, 871, 314, 1556, 14068, 16902, 1029, 6899, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [41]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4328 [00:00<?, ? examples/s]

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

In [42]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [43]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [45]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [46]:
metric = load_metric("seqeval")

C:\Users\grayni\AppData\Local\Temp\ipykernel_18872\152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [47]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

C:\Python310\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python310\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Python310\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [48]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [49]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [50]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.614814519882202,
 'eval_precision': 0.0162876784769963,
 'eval_recall': 0.09378806333739342,
 'eval_f1': 0.027755249166441384,
 'eval_accuracy': 0.07054496823804747,
 'eval_runtime': 4.6115,
 'eval_samples_per_second': 104.304,
 'eval_steps_per_second': 6.722}

In [51]:
for param in model.bert.parameters():
    param.requires_grad = False

In [52]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[ 0.0020,  0.0191, -0.0035,  ...,  0.0209,  0.0066,  0.0142],
        [-0.0308,  0.0264,  0.0095,  ..., -0.0042, -0.0401,  0.0009],
        [-0.0140,  0.0128, -0.0170,  ...,  0.0098, -0.0225,  0.0293],
        ...,
        [ 0.0412,  0.0246, -0.0099,  ...,  0.0154, -0.0049,  0.0187],
        [ 0.0347, -0.0499,  0.0164,  ..., -0.0088, -0.0410,  0.0086],
        [-0.0183, -0.0075, -0.0209,  ...,  0.0136,  0.0019, -0.0092]],
       device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


In [53]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=2710, training_loss=1.1229413697640394, metrics={'train_runtime': 40.7584, 'train_samples_per_second': 1061.868, 'train_steps_per_second': 66.489, 'total_flos': 40223975493360.0, 'train_loss': 1.1229413697640394, 'epoch': 10.0})

In [55]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [56]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [57]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=5420, training_loss=0.3395766226567905, metrics={'train_runtime': 205.478, 'train_samples_per_second': 421.262, 'train_steps_per_second': 26.378, 'total_flos': 80410645167600.0, 'train_loss': 0.3395766226567905, 'epoch': 20.0})

In [59]:
trainer.evaluate()

{'eval_loss': 0.3497312366962433,
 'eval_precision': 0.6329268292682927,
 'eval_recall': 0.6321559074299634,
 'eval_f1': 0.6325411334552102,
 'eval_accuracy': 0.8992811768639251,
 'eval_runtime': 0.5396,
 'eval_samples_per_second': 891.375,
 'eval_steps_per_second': 57.448,
 'epoch': 20.0}

In [60]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ADR': {'precision': 0.39267015706806285,
  'recall': 0.2830188679245283,
  'f1': 0.32894736842105265,
  'number': 265},
 'DI': {'precision': 0.44623655913978494,
  'recall': 0.5460526315789473,
  'f1': 0.49112426035502954,
  'number': 456},
 'Drugclass': {'precision': 0.803680981595092,
  'recall': 0.8136645962732919,
  'f1': 0.8086419753086419,
  'number': 161},
 'Drugform': {'precision': 0.8383458646616542,
  'recall': 0.8352059925093633,
  'f1': 0.8367729831144465,
  'number': 267},
 'Drugname': {'precision': 0.7833698030634574,
  'recall': 0.8927680798004988,
  'f1': 0.8344988344988346,
  'number': 401},
 'Finding': {'precision': 0.4,
  'recall': 0.021739130434782608,
  'f1': 0.041237113402061855,
  'number': 92},
 'overall_precision': 0.6329268292682927,
 'overall_recall': 0.6321559074299634,
 'overall_f1': 0.6325411334552102,
 'overall_accuracy': 0.8992811768639251}

In [61]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [62]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,9616,19,60,9,22,41,0,5,19,0,0,0,0
B-ADR,93,79,74,7,1,0,3,5,3,0,0,0,0
B-DI,154,17,266,0,8,4,0,0,7,0,0,0,0
B-Drugclass,21,1,4,131,0,4,0,0,0,0,0,0,0
B-Drugform,32,0,2,2,224,7,0,0,0,0,0,0,0
B-Drugname,17,0,7,3,4,367,0,0,3,0,0,0,0
B-Finding,21,23,32,2,4,5,2,0,3,0,0,0,0
I-ADR,110,13,18,1,1,0,0,38,21,0,0,0,0
I-DI,142,13,54,1,0,0,0,6,36,0,0,0,0
I-Drugclass,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin\\tokenizer_config.json',
 'ner_bert.bin\\special_tokens_map.json',
 'ner_bert.bin\\vocab.txt',
 'ner_bert.bin\\added_tokens.json',
 'ner_bert.bin\\tokenizer.json')

In [64]:
import torch

In [65]:
text = ' '.join(ner_train[8]['tokens'])
text = ' '.join(ner_test[4]['tokens'])
text

'Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .'

In [66]:
import torch

In [67]:
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 29, 13])

In [68]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
О               O         
##хо            O         
##тно           O         
при             O         
##мен           O         
##я             O         
##ю             O         
его             O         
при             O         
борьбе          O         
с               O         
нас             B-DI      
##мор           B-DI      
##ком           B-DI      
,               O         
что             O         
в               O         
м               O         
##ое            O         
##м             O         
случае          O         
я               O         
##вление        O         
очень           O         
часто           O         
##е             O         
.               O         
[SEP]           O         


In [69]:
from transformers import pipeline

In [70]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [71]:
print(text)
print(pipe(text))

Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .
[{'entity_group': 'DI', 'score': 0.7875536, 'word': 'насморком', 'start': 33, 'end': 42}]
